In [1]:
import os
import matplotlib.pyplot as plt

### Experiment 1

In [43]:
''' latex format
    \textsf{Референсная модель} & $\mathcal{L}_{total}$ & 0.4894 &   0.3775 &  0.4263 &     0.4824 &    0.3683 & 0 \\
    {} & $\mathcal{L}_{eng}$ & 0.4894 &   0.3775 &  0.4263 &     0.4824 &    0.3683 & 0 \\
    {} & $\mathcal{L}_{pos}$ & 0.4894 &   0.3775 &  0.4263 &     0.4824 &    0.3683 & 0 \\
'''

dir = "raw/exp1"
paths = {
    "eng": "RMSE_E_eng.txt",
    "pos": "RMSE_pos.txt",
    "total": "val_total.txt",
}

shapes = set()

per_model = {}
for loss, path in paths.items():
    with open(os.path.join(dir, path), 'r') as f:
        for line in f:
            _, exp_name, value = line.strip().split()
            _, model, shape = exp_name.split('_')
            if model not in per_model:
                per_model[model] = {}
            if loss not in per_model[model]:
                per_model[model][loss] = {}
            per_model[model][loss][shape] = float(value)

            shapes.add(shape)


models = [
    "SumModel",
    "LinearModel",
    "MyResnet18",
    "MyCNN",
    "MyViT",
]

for model in models:
    data = per_model[model]
    print("\\textsf{", model, "} & ", "$\mathcal{L}_{\mathsf{total}}$", sep='', end=' ')
    for shape in shapes:
        # print(f"& {data['total'][shape]:0.4f}", end=' ')
        print("& " "$\\mathsf{", f"{data['total'][shape]:0.4f}", "}$", sep='', end=' ')
    print("\\\\")
    print("{} & $\mathcal{L}_{\mathsf{eng}}$", end=' ')
    for shape in shapes:
        # print(f"& {data['eng'][shape]:0.4f}", end=' ')
        print("& " "$\\mathsf{", f"{data['eng'][shape]:0.4f}", "}$", sep='', end=' ')
    print("\\\\")
    print("{} & $\mathcal{L}_{\mathsf{pos}}$", end=' ')
    for shape in shapes:
        # print(f"& {data['pos'][shape]:0.4f}", end=' ')
        print("& " "$\\mathsf{", f"{data['pos'][shape]:0.4f}", "}$", sep='', end=' ')
    print("\\\\")
    if model != models[-1]:
        print("\\midrule")

\textsf{SumModel} & $\mathcal{L}_{\mathsf{total}}$ & $\mathsf{0.1560}$ & $\mathsf{0.1563}$ & $\mathsf{0.1575}$ & $\mathsf{0.1555}$ & $\mathsf{0.1561}$ & $\mathsf{0.1549}$ \\
{} & $\mathcal{L}_{\mathsf{eng}}$ & $\mathsf{0.0220}$ & $\mathsf{0.0226}$ & $\mathsf{0.0229}$ & $\mathsf{0.0221}$ & $\mathsf{0.0222}$ & $\mathsf{0.0213}$ \\
{} & $\mathcal{L}_{\mathsf{pos}}$ & $\mathsf{0.2901}$ & $\mathsf{0.2901}$ & $\mathsf{0.2924}$ & $\mathsf{0.2891}$ & $\mathsf{0.2901}$ & $\mathsf{0.2887}$ \\
\midrule
\textsf{LinearModel} & $\mathcal{L}_{\mathsf{total}}$ & $\mathsf{0.0748}$ & $\mathsf{0.0741}$ & $\mathsf{0.0745}$ & $\mathsf{0.0738}$ & $\mathsf{0.0751}$ & $\mathsf{0.0728}$ \\
{} & $\mathcal{L}_{\mathsf{eng}}$ & $\mathsf{0.0221}$ & $\mathsf{0.0226}$ & $\mathsf{0.0231}$ & $\mathsf{0.0221}$ & $\mathsf{0.0223}$ & $\mathsf{0.0213}$ \\
{} & $\mathcal{L}_{\mathsf{pos}}$ & $\mathsf{0.1278}$ & $\mathsf{0.1257}$ & $\mathsf{0.1264}$ & $\mathsf{0.1258}$ & $\mathsf{0.1282}$ & $\mathsf{0.1244}$ \\
\midrule
\te